# Transformación de los datos 

In [1]:
# !pip install pyarrow

In [1]:
# Importar librerías
import pandas as pd
import os
import warnings
import re

In [2]:
# Crear carpeta para almacenar datos Tidy
if not os.path.exists('Tidy'):
    os.mkdir("Tidy")

## SIAP

### Unión de los datos

In [3]:
# Arreglos con valores de años, meses y cultivos
anio_arr = pd.read_csv('./Catalogos_API_SIAP/Anio_Agricola.csv').ID.to_list()
cultivo_arr = pd.read_csv('./Catalogos_API_SIAP/Cultivo.csv').ID.to_list()
mes_arr = pd.read_csv('./Catalogos_API_SIAP/Mes_Agricola.csv').ID.to_list()
cultivo_arr.remove(0) # 0: Todos

In [90]:
# Dataframe vacío con las columnas de nuestros datos
columnas = ["Entidad", "Municipio", "Superficie Sembrada", "Superficie Cosechada", "Superficie Siniestrada", "Produccion", "Rendimiento","Anio", "Mes", "Cultivo"]
df_raw = pd.DataFrame(columns=columnas)

total = 0
# Iteración sobre CSV's para llenar DataFrame
for anio in anio_arr:
    for i in range(1,13):
        df_aux = pd.read_csv("./raw/SIAP/"+str(anio)+"/Avance_Agricola_"+str(anio)+ "_" + str(i) + '.csv')
        df_raw = pd.concat([df_raw, df_aux], axis=0)
        total += df_aux.shape[0]


C:\Users\jcbar\AppData\Local\Temp\ipykernel_4416\2230714408.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_raw = pd.concat([df_raw, df_aux], axis=0)


In [83]:
# Dimensión de los datos
print(f"Dimensión de los datos: {df_raw.shape}")

Dimensión de los datos: (1154774, 10)


In [84]:
# 5 filas ejemplos
df_raw.sample(n=5)

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,Anio,Mes,Cultivo
363075,Guanajuato,San Francisco del Rincón,7.00,0.00,0.00,0.00,0,2021,1,212
968386,Durango,San Dimas,24.00,0.00,0.00,0.00,0,2018,1,392
349923,Puebla,Acatlán,10.50,10.00,0.00,90.79,9.08,2022,12,230
1098262,Guanajuato,San Felipe,"5,250.00",0.00,0.00,0.00,0,2018,10,6
998923,Oaxaca,Santiago Pinotepa Nacional,79.00,42.00,0.00,176.89,4.21,2018,4,89


In [97]:
# Para poder almacenar los datos en formato parquet debemos convertir la columna "Superficie Siniestrada" en flotante
# Sin embargo, esta columna contiene datos string y flotantes 
strings = df_raw['Superficie Siniestrada'].apply(lambda x: isinstance(x, str)); # Identificar elementos de la columna que son strings
df_raw['Superficie Siniestrada'][strings] = df_raw['Superficie Siniestrada'][strings].str.replace(',', '').astype(float);  # Eliminar comas en estos elementos
df_raw['Superficie Siniestrada'] = df_raw['Superficie Siniestrada'].astype(float)   # Convertir elementos en float

In [92]:
# Si aún no existe, guardamos el DataFrame en un parquet
if not os.path.exists('raw/SIAP/Avance_Agricola_Historico.parquet'):
    name_csv = "Avance_Agricola_Historico"
    ruta_completa = os.path.join("raw/SIAP/",name_csv)
    df_raw.to_parquet(ruta_completa + '.parquet', index=False)

### Decodificación de variables

In [93]:
# Cargar parquet
df = pd.read_parquet('./raw/SIAP/Avance_Agricola_Historico.parquet')

In [94]:
# Cargar catálogos
catalogo_cultivos = pd.read_csv('./Catalogos_API_SIAP/Cultivo.csv')  # Cultivos
catalogo_mes = pd.read_csv('./Catalogos_API_SIAP/Mes_Agricola.csv')  # Meses

# Convertir los id's en keys de diccionario
catalogo_cultivos = catalogo_cultivos.set_index('ID')['Name'].to_dict()
catalogo_mes = catalogo_mes.set_index('ID')['Name'].to_dict()

# Mapear los valores de nuestras columnas con respecto a estos catálgoso
df['Cultivo'] = df['Cultivo'].map(catalogo_cultivos)
df['Mes'] = df['Mes'].map(catalogo_mes)


In [95]:
# 5 filas ejemplos
df.sample(n=5)

,Entidad,Municipio,Superficie Sembrada,Superficie Cosechada,Superficie Siniestrada,Produccion,Rendimiento,Anio,Mes,Cultivo
637709,Sonora,Guaymas,198.00,198.00,0.0,389.00,1.96,2020,Junio,Garbanzo grano
720511,Jalisco,Zacoalco de Torres,201.61,201.61,0.0,"3,885.33",19.27,2020,Octubre,Tomate verde
680414,Guerrero,Zihuatanejo de Azueta,102.00,92.00,0.0,"2,922.45",31.77,2020,Agosto,Sandía
140328,Oaxaca,San Andrés Ixtlahuaca,384.00,6.00,0.0,17.94,2.99,2023,Septiembre,Maíz grano
503881,Michoacán,Cotija,6.00,6.00,0.0,117.50,19.58,2021,Septiembre,Zarzamora


In [96]:
# Crear parquet con los valores decodificados
if not os.path.exists('./Tidy/SIAP.parquet'):
    df.to_parquet('./Tidy/SIAP.parquet', index=False)

## SNIIM

### Unión de los datos

In [232]:
# Crea una lista para almacenar los DataFrames de los archivos CSV
dataframes = []

# Itera a través de los archivos en la ruta
for archivo in os.listdir("./raw/SNIIM/"):
    if archivo.endswith(".csv"):
        # Lee cada archivo CSV y lo convierte en un DataFrame
        archivo_csv = os.path.join('./raw/SNIIM/' + str(archivo))
        if os.path.getsize(archivo_csv) > 0:
            try:
                df_sniim = pd.read_csv(archivo_csv)
                dataframes.append(df_sniim)
            except pd.errors.EmptyDataError:
                print(f"El archivo {archivo} está vacío y se omitirá.")

El archivo sniim_product_aguacate_fuerte_primera.csv está vacío y se omitirá.
El archivo sniim_product_aguacate_hass_flor_vieja_primera.csv está vacío y se omitirá.
El archivo sniim_product_melón_cantaloupe_#_16_primera.csv está vacío y se omitirá.
El archivo sniim_product_melón_cantaloupe_#_45_primera.csv está vacío y se omitirá.
El archivo sniim_product_melón_cantaloupe_#_48_primera.csv está vacío y se omitirá.
El archivo sniim_product_papa_marciana_primera.csv está vacío y se omitirá.
El archivo sniim_product_plátano_enanogigante_primera.csv está vacío y se omitirá.
El archivo sniim_product_santa_lucía_girasol.csv está vacío y se omitirá.
El archivo sniim_product_santa_lucía_vegetal_mixto.csv está vacío y se omitirá.
El archivo sniim_product_santísima_vegetal_mixto.csv está vacío y se omitirá.
El archivo sniim_product_sarita_cártamo.csv está vacío y se omitirá.
El archivo sniim_product_sarita_maíz.csv está vacío y se omitirá.
El archivo sniim_product_sarita_vegetal_mixto.csv está va

In [233]:
# Combina todos los DataFrames en uno solo
df_completo = pd.concat(dataframes, ignore_index=True)
# Muestra de 5 filas aleatorias del dataframe
df_completo.sample(n=5)

,fecha,presentacion,origen,destino,precio_min,precio_max,precio_frec,obs,name
171120,08/05/2019,Kilogramo,Sinaloa,"Coahuila: Central de Abasto de La Laguna, Torreón",20.00,21.0,20.50,NaN,chile chilaca_primera
328492,02/10/2020,Caja de 12 kg.,México,DF: Central de Abasto de Iztapalapa DF,19.17,21.67,20.83,NaN,durazno amarillo_primera
420945,17/09/2020,Kilogramo,Guanajuato,Aguascalientes: Centro Comercial Agropecuario ...,6.00,8.0,7.00,NaN,jícama_primera
225228,17/06/2022,Kilogramo,Aguascalientes,Aguascalientes: Centro Comercial Agropecuario ...,110.00,130.0,120.00,NaN,chile pasilla_primera
860337,18/07/2023,Caja de 9 kg.,Distrito Federal,"Campeche: Mercado ""Pedro Sáinz de Baranda"", Ca...",55.56,61.11,61.11,NaN,uva sin semilla_primera


In [234]:
# Dimensión de los datos
print(f"Dimensión de los datos: {df_completo.shape}")

Dimensión de los datos: (900763, 9)


In [235]:
# Renombrar las columnas para que sigan el mismo formato que los datos del SIAP
nuevos_nombres = {
    "fecha": "Fecha",
    "presentacion": "Presentacion",
    "origen": "Origen",
    "destino": "Destino",
    "precio_min": "Precio Minimo",
    "precio_max": "Precio Maximo",
    "precio_frec": "Precio Frecuente",
    "obs": "Observacion",
    "name":"Cultivo"
}

# Aplicar el renombramiento al df_completo
df_completo = df_completo.rename(columns=nuevos_nombres)

In [236]:
# Muestra de 5 filas aleatorias del dataframe
df_completo.sample(n=5)

,Fecha,Presentacion,Origen,Destino,Precio Minimo,Precio Maximo,Precio Frecuente,Observacion,Cultivo
51936,01/12/2022,Manojo de 4 kg.,Puebla,Chiapas: Central de Abasto de Tuxtla Gutiérrez,10.00,15.0,10.00,NaN,betabel_primera
796001,09/09/2020,Caja de 13 kg.,Sinaloa,Aguascalientes: Centro Comercial Agropecuario ...,16.92,18.46,17.69,NaN,tomate bola_primera
152265,18/10/2022,Kilogramo,Sonora,Chihuahua: Mercado de Abasto de Cd. Juárez,40.00,40.0,40.00,NaN,chile california_primera
819015,26/09/2022,Kilogramo,Tamaulipas,Aguascalientes: Centro Comercial Agropecuario ...,15.00,17.0,16.00,NaN,toronja roja_primera
73096,29/09/2020,Caja de 15 kg.,Aguascalientes,Aguascalientes: Centro Comercial Agropecuario ...,11.33,12.67,12.00,NaN,calabacita italiana_primera


El dataframe no puede almacenarse como parquet porque:
- La columna "Precio Maximo" contiene floats y strings
- La columna "Observacion" contiene inconsistencias

In [237]:
# Tipo de dato de cada columna
df_completo.dtypes

Fecha                object
Presentacion         object
Origen               object
Destino              object
Precio Minimo       float64
Precio Maximo        object
Precio Frecuente    float64
Observacion          object
Cultivo              object
dtype: object

In [238]:
# "Precio Maximo" de object a float
strings = df_completo['Precio Maximo'].apply(lambda x: isinstance(x, str)); # Identificar elementos de la columna que son strings
df_completo['Precio Maximo'][strings] = df_completo['Precio Maximo'][strings].str.replace(',', '').astype(float);  # Eliminar comas en estos elementos
df_completo['Precio Maximo'] = df_completo['Precio Maximo'].astype(float)   # Convertir elementos en float
# Nuevo tipo de dato de cada columna
df_completo.dtypes

C:\Users\jcbar\AppData\Local\Temp\ipykernel_15524\4129408368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_completo['Precio Maximo'][strings] = df_completo['Precio Maximo'][strings].str.replace(',', '').astype(float);  # Eliminar comas en estos elementos


Fecha                object
Presentacion         object
Origen               object
Destino              object
Precio Minimo       float64
Precio Maximo       float64
Precio Frecuente    float64
Observacion          object
Cultivo              object
dtype: object

In [239]:
# "Observacion" a str
df_completo['Observacion'] = df_completo['Observacion'].astype(str)

In [240]:
# Almacenamos el archivo como parquet 
if not os.path.exists('./Tidy/SNIIM.parquet'):
    df_completo.to_parquet('./Tidy/SNIIM.parquet', index=False)